In [1]:
from datasets import load_dataset

stoic = load_dataset('jamescalam/stoic-corpus', split='train')
stoic

Using custom data configuration jamescalam--stoic-corpus-0a7f055fafc72e16


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 184.74it/s]

Dataset json downloaded and prepared to /Users/jamesbriggs/.cache/huggingface/datasets/json/jamescalam--stoic-corpus-0a7f055fafc72e16/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'text', 'source'],
    num_rows: 1647
})

In [2]:
stoic[0]

{'id': '0',
 'text': 'From my grandfather Verus I learned good morals and the government of my temper.',
 'source': 'meditations'}

## Query Generation


We use a query generation model to generate *(query, doc)* pairs, where a doc is a single paragraph from the samples we produced above. For this, we use a T5 query generation model.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 'doc2query/all-t5-base-v1'

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)